In [1]:
# Dataset: source code for 'requests' python libraray
text = open("data/requests.txt").read()

In [2]:
import numpy as np

# sorted list of all unique characters in the text
chars = sorted(list(set(text)))
vocab_size = len(chars)

# string-to-integer mapping
stoi = { char:i for i,char in enumerate(chars) }

# integer-to-string mapping
itos = { i:char for i,char in enumerate(chars) }

# lookup functions for the mappings
encode = lambda s: [stoi[c] for c in s]
decode = lambda l: ''.join([itos[i] for i in l])

# encode the entire text file and convert to a numpy array
data = np.array(encode(text))

In [3]:
import numpy as np

# Each character has weights of a 32 long vector, defined by n_embed (embedding dimension)
n_embd = 32

# Initialize embedding & unembedding matrix
embedding_matrix = np.random.randn(vocab_size, n_embd)
unembedding_matrix = np.random.randn(n_embd, vocab_size)



In [4]:
# Standard expansion factor of four
ffwd_expansion_factor = 4

# Initialize hidden layer and output layer
# Use Kaiming init to intelligently scale the layer's random weights
W1 = np.random.randn(n_embd, n_embd * ffwd_expansion_factor) * np.sqrt(2.0 / n_embd)
W2 = np.random.randn(n_embd * ffwd_expansion_factor, n_embd) * np.sqrt(2.0 / n_embd)
 

In [93]:
def cross_entropy_loss(y_pred, y_true):

    # Add a small epsilon to the prediction to avoid log(0), which is undefined.
    epsilon = 1e-9
    
    # cross-entropy formula
    loss = -np.sum(y_true * np.log(y_pred + epsilon))
    
    return loss

In [137]:
class Model:
    def __init__(self,embedding_matrix, unembedding_matrix, W1, W2):
        self.embedding_matrix = embedding_matrix
        self.unembedding_matrix = unembedding_matrix
        self.W1 = W1
        self.W2 = W2
        self.cache = {} # A dictionary to store forward pass values


    def forward(self, x_batch):

        self.cache['x_batch'] = x_batch

        # Output shape: (B, T, n_embd)
        embd = self.embedding_matrix[x_batch]
        self.cache['embd'] = embd

        hidden = embd @ self.W1
        self.cache['hidden'] = hidden
        
        hidden_activated = np.maximum(0, hidden)
        self.cache['hidden_activated'] = hidden_activated 
        
        processed_vectors = hidden_activated @ self.W2 # Shape: (B, T, n_embd)
        self.cache['processed_vectors'] = processed_vectors

        # Final projection to logits
        logits = processed_vectors @ self.unembedding_matrix
        
        return logits

    def pred (self, x):

        logits = self.forward(x)

        ## Apply softmax function to logits
        stable_logits = logits - np.max(logits) # This ensures the largest logit is 0
        raw_preds = np.exp(stable_logits) / np.sum(np.exp(stable_logits))        
        preds = {}

        for idx, raw_pred in enumerate(raw_preds):

            preds[itos[idx]] = raw_pred
        
        return preds

    def loss (self, logits, y_batch):

        # Get the dimensions for indexing
        B, T, C = logits.shape

        # Stable softmax
        max_logits = np.max(logits, axis=-1, keepdims=True)
        stable_logits = logits - max_logits
        exp_logits = np.exp(stable_logits)
        probabilities = exp_logits / np.sum(exp_logits, axis=-1, keepdims=True)
    
        # Get the probabilities for the correct target characters using efficient indexing
        correct_char_probs = probabilities[np.arange(B)[:, None], np.arange(T), y_batch]

        # Calculate negative log likelihood
        loss_array = -np.log(correct_char_probs + 1e-9)
    
        # Average the loss over the whole batch to get a single number
        mean_loss = np.mean(loss_array)
    
        # Return probabilities because they are the starting point for backpropagation
        return mean_loss, probabilities

    
    # Calculates the gradients for a specific layer and it's resulting vector
    def linear_backward(d_output, W, x_from_cache):

        # d_W = x.T @ dy
        # d_x = dy @ W.T

        d_x = d_output @ W.T

        x_reshaped, dy_reshaped = x_from_cache.reshape(-1, x_from_), d_output.reshape(-1, C)
        d_W = x_reshaped.T @ dy_reshaped

        return d_W, d_x


    def backward (self, d_logits):

        print(self.unembedding_matrix.grad)
        
        # unembedding layer
        grad_unembed, d_processed = self.linear_backward(d_logits, self.unembedding_matrix, self.cache['processed_vectors'])

        

        


In [138]:
model = Model(embedding_matrix, unembedding_matrix, W1, W2)
# Get next character predictions for 'd'
logits = model.forward([stoi['d'],stoi['d']])

/var/folders/8k/dg_kyxnn5wn4nbhpwfq40c2r0000gn/T/ipykernel_19714/1937987906.py:18: RuntimeWarning: divide by zero encountered in matmul
  hidden = embd @ self.W1
/var/folders/8k/dg_kyxnn5wn4nbhpwfq40c2r0000gn/T/ipykernel_19714/1937987906.py:18: RuntimeWarning: overflow encountered in matmul
  hidden = embd @ self.W1
/var/folders/8k/dg_kyxnn5wn4nbhpwfq40c2r0000gn/T/ipykernel_19714/1937987906.py:18: RuntimeWarning: invalid value encountered in matmul
  hidden = embd @ self.W1
/var/folders/8k/dg_kyxnn5wn4nbhpwfq40c2r0000gn/T/ipykernel_19714/1937987906.py:24: RuntimeWarning: divide by zero encountered in matmul
  processed_vectors = hidden_activated @ self.W2 # Shape: (B, T, n_embd)
/var/folders/8k/dg_kyxnn5wn4nbhpwfq40c2r0000gn/T/ipykernel_19714/1937987906.py:24: RuntimeWarning: overflow encountered in matmul
  processed_vectors = hidden_activated @ self.W2 # Shape: (B, T, n_embd)
/var/folders/8k/dg_kyxnn5wn4nbhpwfq40c2r0000gn/T/ipykernel_19714/1937987906.py:24: RuntimeWarning: invalid va

In [109]:
import random

def get_batch(data, batch_size, block_size):

    x_batch = []
    y_batch = []

    # Generate batchs 
    for block in [0] * batch_size:

        # Get random range in datast of size=block_size
        slice_idx = random.randrange(0, len(data) - block_size)
        x_batch.append(data[slice_idx:slice_idx+block_size])
        y_batch.append(data[slice_idx+1:slice_idx+block_size+1])

    return np.array(x_batch), np.array(y_batch)



In [139]:
# Training hyperparameters
max_iters = 5000
learning_rate = 1e-3 # A common starting point for learning rate
eval_interval = 500  # How often we'll print the loss
batch_size = 4
block_size = 8

# Training loop
for step in range(max_iters):
    
    # Get a mini-batch of data
    x_batch, y_batch = get_batch(data, batch_size, block_size)

    
    # Calculate loss and probabilites
    logits = model.forward(x_batch)
    loss_initial, probabilities = model.loss(logits, y_batch)

    print(f"Loss: {loss_initial}")

    one_hot_array = np.eye(vocab_size)[y_batch]

    initial_gradient = probabilities - one_hot_array
          
    # Backward Pass
    model.backward(initial_gradient)
    
    # Optimizer step (Updated weights with gradients)
    for param in model.parameters():
        param -= learning_rate * param.grad 
        
    # Print out the loss periodically
    if step % eval_interval == 0:
        print(f"Step {step}: Training Loss = {loss}")



Loss: 19.98337779353372


/var/folders/8k/dg_kyxnn5wn4nbhpwfq40c2r0000gn/T/ipykernel_19714/1937987906.py:18: RuntimeWarning: divide by zero encountered in matmul
  hidden = embd @ self.W1
/var/folders/8k/dg_kyxnn5wn4nbhpwfq40c2r0000gn/T/ipykernel_19714/1937987906.py:18: RuntimeWarning: overflow encountered in matmul
  hidden = embd @ self.W1
/var/folders/8k/dg_kyxnn5wn4nbhpwfq40c2r0000gn/T/ipykernel_19714/1937987906.py:18: RuntimeWarning: invalid value encountered in matmul
  hidden = embd @ self.W1
/var/folders/8k/dg_kyxnn5wn4nbhpwfq40c2r0000gn/T/ipykernel_19714/1937987906.py:24: RuntimeWarning: divide by zero encountered in matmul
  processed_vectors = hidden_activated @ self.W2 # Shape: (B, T, n_embd)
/var/folders/8k/dg_kyxnn5wn4nbhpwfq40c2r0000gn/T/ipykernel_19714/1937987906.py:24: RuntimeWarning: overflow encountered in matmul
  processed_vectors = hidden_activated @ self.W2 # Shape: (B, T, n_embd)
/var/folders/8k/dg_kyxnn5wn4nbhpwfq40c2r0000gn/T/ipykernel_19714/1937987906.py:24: RuntimeWarning: invalid va

AttributeError: 'numpy.ndarray' object has no attribute 'grad'